In [ ]:
import logging
import requests
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForPreTraining
from io import BytesIO
from huggingface_hub import login


login('API token') - Ваш API токен для доступа к модели
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)


processor = AutoProcessor.from_pretrained("google/paligemma-3b-mix-224")
model = AutoModelForPreTraining.from_pretrained("google/paligemma-3b-mix-224")

def process_image(image_data):
    prompt = "Describe the clothing in this image. Include type, color, style, and any patterns or unique features."
    model_inputs = processor(text=prompt, images=image_data, return_tensors="pt").to("cpu")
    input_len = model_inputs["input_ids"].shape[-1]

    with torch.no_grad():
        generation = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, top_k=50, top_p=0.95)
        generation = generation[0][input_len:]
        decoded_description = processor.decode(generation, skip_special_tokens=True)

    return decoded_description

def describe_image(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        description = process_image(img)
        return description
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return "Could not process the image from the link. Please check the URL."
    except Exception as e:
        logging.error(f"Error processing image: {e}")
        return "An error occurred while processing the image."



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
image_url = input("Введите ссылку на изображение: ")
description = describe_image(image_url)
print(f"Описание: {description}")

Введите ссылку на изображение:  https://cdn.100sp.ru/pictures/172762609


Описание: blue checkered shirt
